In [317]:
import pandas as pd
import numpy as np
import math
import json

In [318]:
from chardet.universaldetector import UniversalDetector
detector = UniversalDetector()
with open('cities.csv', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
print(detector.result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [319]:
cities = pd.read_csv('cities.csv', encoding = 'utf-8')
cities.sample(5)

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
331,612960.0,обл,Кировская,р-н,Вятскополянский,г,Вятские Поляны,NaN,NaN,4300800100000,...,4: город,1,33404000000,3.370400e+07,4307,UTC+3,56.228606,51.061467,Приволжский,35159
777,446450.0,обл,Самарская,NaN,NaN,г,Похвистнево,NaN,NaN,6300000900000,...,4: город,0,36427000000,3.672700e+10,6372,UTC+4,53.649820,52.123552,Приволжский,28181
504,430000.0,Респ,Мордовия,NaN,NaN,г,Саранск,NaN,NaN,1300000100000,...,4: город,2,89401000000,8.970100e+07,1300,UTC+3,54.180760,45.186226,Приволжский,297425
904,393670.0,обл,Тамбовская,р-н,Жердевский,г,Жердевка,NaN,NaN,6800400100000,...,4: город,1,68204501000,6.860410e+10,6820,UTC+3,51.842178,41.461741,Центральный,15211
943,171571.0,обл,Тверская,р-н,Калязинский,г,Калязин,NaN,NaN,6901100100000,...,4: город,1,28222501000,2.862210e+07,6910,UTC+3,57.257947,37.782125,Центральный,13870


In [320]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Индекс                             1108 non-null   float64
 1   Тип региона                        1112 non-null   object 
 2   Регион                             1112 non-null   object 
 3   Тип района                         629 non-null    object 
 4   Район                              629 non-null    object 
 5   Тип города                         1108 non-null   object 
 6   Город                              1108 non-null   object 
 7   Тип н/п                            2 non-null      object 
 8   Н/п                                2 non-null      object 
 9   Код КЛАДР                          1112 non-null   int64  
 10  Код ФИАС                           1112 non-null   object 
 11  Уровень по ФИАС                    1112 non-null   objec

In [321]:
cities.columns

Index(['Индекс', 'Тип региона', 'Регион', 'Тип района', 'Район', 'Тип города',
       'Город', 'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
       'Код ИФНС', 'Часовой пояс', 'Широта', 'Долгота', 'Федеральный округ',
       'Население'],
      dtype='object')

In [322]:
cities.drop(['Индекс', 'Тип региона', 'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
       'Код ИФНС', 'Федеральный округ'], axis = 1, inplace = True)

In [323]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Регион        1112 non-null   object 
 1   Тип района    629 non-null    object 
 2   Район         629 non-null    object 
 3   Тип города    1108 non-null   object 
 4   Город         1108 non-null   object 
 5   Часовой пояс  1112 non-null   object 
 6   Широта        1112 non-null   float64
 7   Долгота       1112 non-null   float64
 8   Население     1112 non-null   object 
dtypes: float64(2), object(7)
memory usage: 78.3+ KB


In [324]:
cities[cities['Город'] == 'Казань']

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население
924,Татарстан,NaN,NaN,г,Казань,UTC+3,55.794388,49.111531,1216965


In [325]:
cities['Часовой пояс'] = cities['Часовой пояс'].str.replace('UTC+', '')#удаляю UTC из часового пояса

In [326]:
cities['Часовой пояс'] = cities['Часовой пояс'].str.replace(' ', '')

In [327]:
cities['Часовой пояс'] = cities['Часовой пояс'].apply(lambda x: np.int(x))

In [328]:
cities['Часовой пояс'].unique()

array([ 3,  7,  9,  4,  5,  8, 10,  2, 12, 11,  6], dtype=int64)

In [ ]:
cities[cities['Город'] == Белогорск]['Часовой пояс']

In [330]:
for i in cities['Город']:
    print(i)
    int(cities[cities['Город'] == i]['Часовой пояс'].values)+2.5

Адыгейск
Майкоп
Горно-Алтайск
Алейск
Барнаул
Белокуриха
Бийск
Горняк
Заринск
Змеиногорск
Камень-на-Оби
Новоалтайск
Рубцовск
Славгород
Яровое
Белогорск


TypeError: only size-1 arrays can be converted to Python scalars

In [191]:
#Отсекаем часовые пояса +- 2 часа
near_Kazan = cities[cities['Часовой пояс'].between(float(cities[cities['Город'] == 'Самара']['Часовой пояс'].values)-2.5,
                                            float(cities[cities['Город'] == 'Самара']['Часовой пояс'].values)+2.5,
                                            inclusive = True)]

In [192]:
# Отсекаем города по Долготе
near_Kazan = near_Kazan[near_Kazan['Долгота'].between(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Долгота'].values)-1,
                                            float(near_Kazan[near_Kazan['Город'] == 'Самара']['Долгота'].values)+1,
                                            inclusive = True)]

In [205]:
math.cos(round(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Широта'].values), 6) * 0.017)

0.618221918105964

In [193]:
max_latitude = 40000/360 * math.cos(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Широта'].values))
print(max_latitude)

-108.62567738421929


In [189]:
latitude = round(111.111/(max_latitude/60)/60, 6)
latitude

-1.02288

In [187]:
near_Kazan = near_Kazan[near_Kazan['Широта'].between(float(near_Kazan[near_Kazan['Город'] == 'Самара']
                                                           ['Широта'].values)-latitude,
                                                     float(near_Kazan[near_Kazan['Город'] == 'Самара']
                                                           ['Широта'].values)+latitude,
                                            inclusive = True)]

In [188]:
near_Kazan

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население


In [165]:
near_sity_s = dict()

near_sity_s['Казань'] = list()

for i in near_Kazan['Город']:
    near_sity_s['Казань'].append(i)

print(near_sity_s) 

{'Казань': []}
{'Казань': ['Уржум', 'Волжск', 'Арск', 'Болгар', 'Буинск', 'Зеленодольск', 'Иннополис', 'Казань', 'Лаишево', 'Тетюши', 'Козловка']}


In [219]:
sev_city = ['Казань', 'Самара', 'Ульяновск', 'Альметьевск']

In [233]:
several_city = cities.query('Город in ["Казань","Самара"]')

In [315]:
def near_city(df, distance):
    near_sity_s = dict() #создаю словарь, что бы добавить города
    
    for i in df['Город']:
        near_sity_s[i] = list()
    
        #Отсекаем часовые пояса +- 2 часа
        near_Kazan = df[df['Часовой пояс'].between(float(df[df['Город'] == i]['Часовой пояс'].values)-2.5,
                                                float(df[df['Город'] == i ]['Часовой пояс'].values)+2.5,
                                                inclusive = True)]
    
        # Отсекаем города по Долготе
        longitude = distance/111.111
        near_Kazan = near_Kazan[near_Kazan['Долгота'].between(float(near_Kazan[near_Kazan['Город'] == i]
                                                            ['Долгота'].values)-longitude,
                                                float(near_Kazan[near_Kazan['Город'] == i]
                                                      ['Долгота'].values)+longitude,
                                                inclusive = True)]
    
        latitude = 40000/360 * math.cos(float(near_Kazan[near_Kazan['Город'] == i]['Широта'].values)*0.017)
        latitude = round(distance/(latitude/60)/60, 6)
    
        # Отсекаем города по Широте
        near_Kazan = near_Kazan[near_Kazan['Широта'].between(float(near_Kazan[near_Kazan['Город'] == i]
                                                               ['Широта'].values)-latitude,
                                                         float(near_Kazan[near_Kazan['Город'] == i]
                                                               ['Широта'].values)+latitude,
                                                inclusive = True)]
    
        for j in near_Kazan['Город']:
            near_sity_s[i].append(j)

    return near_sity_s

Отсекаем часовые пояса +\- 2 часа
Считам длину долготы - разница +\- 111 км
Длина 1 градуса широты 40000\*cos(широты) - 1 градус - длина широты \ 360, в градусе 60 минут - 
   пересчитываем в десятичные доли - 1 градус + 1 мин\60
Находим по разницы широты +\- 1 градус, долготы расчётная разница, список городов
Выводим списко городов

In [329]:
summary = near_city(cities, 50)

TypeError: only size-1 arrays can be converted to Python scalars

In [251]:
print(summary)

{'Самара': ['Жигулевск', 'Кинель', 'Новокуйбышевск', 'Самара', 'Тольятти', 'Чапаевск', 'Нурлат', 'Димитровград'], 'Казань': ['Уржум', 'Волжск', 'Арск', 'Болгар', 'Буинск', 'Зеленодольск', 'Иннополис', 'Казань', 'Лаишево', 'Тетюши', 'Ульяновск', 'Козловка']}


In [252]:
jsonData = json.dumps(summary)

In [253]:
with open("json_near_city.json", "w") as file:
    file.write(jsonData)

In [254]:
with open('json_near_city.json') as f:
   expl = json.load(f)

In [255]:
from pprint import pprint  
pprint(expl)

{'Казань': ['Уржум',
            'Волжск',
            'Арск',
            'Болгар',
            'Буинск',
            'Зеленодольск',
            'Иннополис',
            'Казань',
            'Лаишево',
            'Тетюши',
            'Ульяновск',
            'Козловка'],
 'Самара': ['Жигулевск',
            'Кинель',
            'Новокуйбышевск',
            'Самара',
            'Тольятти',
            'Чапаевск',
            'Нурлат',
            'Димитровград']}
